In [31]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets, linear_model
from sklearn import cross_validation, svm
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import scipy.io
train = scipy.io.loadmat('ReducedImagesForTraining.mat')['images'].T
test = scipy.io.loadmat('ReducedImagesForTesting.mat')['images'].T
print train.shape


def pca(X, wsp):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #rint "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    k = 1; 
    suma = np.sum(D)
    while(np.sum(D[-k:])/suma < wsp):
        k = k+1
    return Y_new[:,-k:], D[-k:], V[:,-k:], k


def PCA(X, k):
    n,m = np.shape(X)
    X_mean = np.mean(X, axis = 0)
    X_std = np.std(X,axis = 0)
    X_standarized = (X - X_mean)/X_std
    S = np.cov(X_standarized.T)
    D,V = np.linalg.eigh(S)
    #print "suma wartosci wlasnych:",np.sum(D)
    Y = np.dot(V.T,X_standarized.T).T
    D_bis = np.matrix(1/np.sqrt(D)).reshape(m,1)
    Y_new = np.multiply(D_bis,Y.T).T
    #print Y_new
    return Y_new[:,-k:]

def nearestPoints1(knownData,testData):
    D1 = np.sum(np.square(knownData),axis = 1,keepdims=True)
    D2 = np.sum(np.square(testData),axis = 1,keepdims=True).T
    M = D1 + D2 - 2. * np.dot(knownData,testData.T)
    #print M
    return np.asarray(np.argmin(M,axis=0))[0]
    

def KN1(knownData, testData, knownClass, testClass):
    error = 0
    nearest = nearestPoints1(knownData,testData)
    #print nearest
    length = testData.shape[0]
    nearestClass = np.empty(length)
    error_list = []
    print length
    for i in range(0,length):
        print i, nearest[i]
        #print nearest[i]
        nearestClass[i] = knownClass[nearest[i]]
        if(not(nearestClass[i] == testClass[i])):
           error += 1
           error_list = np.append(error_list, [i])
    return error, error_list

def nearestPoints(knownData,testData,k):
    D1 = np.sum(np.square(knownData),axis = 1,keepdims=True)
    D2 = np.sum(np.square(testData),axis = 1,keepdims=True).T
    M = D1 + D2 - 2. * np.dot(knownData,testData.T)
    #print M
    return np.asarray(np.argsort(M,axis=0)[:k,:].T)
    

def KNN(knownData, testData, knownClass, testClass,k):
    error = 0
    nearest = nearestPoints(knownData,testData,k)
    #print nearest
    length = testData.shape[0]
    nearestClass = np.empty(length)
    error_list = []
    for i in range(0,length):
        print i,nearest[i]
        nearestClass[i] = np.argmax(np.bincount([knownClass[a] for a in nearest[i]]))
        if(not(nearestClass[i] == testClass[i])):
           error += 1
           error_list = np.append(error_list, [i])
    return error, error_list

from scipy import misc
from IPython.display import Image
from IPython.display import display
def pokaz(i):
    zdj1 = test[i,:]
    plt.figure(figsize=(10,4))
    plt.imshow(zdj1.reshape(60,82).T, cmap = 'gray')
knownClass = [int((k-1)/5) + 1 for k in range(1,251)]
testClass = [int((k-1)/2) + 1 for k in range(1,101)]
#print knownClass
#print testClass

(250, 4920)


In [29]:
train_new , D, V, k = pca(train, 0.95)
print k
print V.shape
print test.shape
test_mean = np.mean(test, axis = 0)
test_std = np.std(test,axis = 0)
test_standarized = (test - test_mean)/test_std
test_new = np.dot(V.T, test_standarized.T).T
print test_new.shape


88
(4920, 88)
(100, 4920)
(100, 88)


In [23]:
error, error_list = KN1(train_new, test_new, knownClass, testClass)
print error, error_list
for i in error_list:
    print testClass[i]
for i in error_list:
    pokaz(i)

[  1   4   5   9  10  14  15  19  20  24  26  29  30  34  35  39  40  44
  45  49  52  54  55  59  60  64  67  69  70  74  77  79  80  84  85  89
  90  94  97  99 100 104 107 109 110 114 115 119 120 124 125 129 130 134
 135 139 142 144 145 149 150 154 156 159 162 164 165 169 170 174 175 179
 182 184 189 189 190 194 195 199 200 204 205 209 210 214 215 219 222 224
 226 229 230 234 239 239 242 244 245 249]
100
0 1
1 4
2 5
3 9
4 10
5 14
6 15
7 19
8 20
9 24
10 26
11 29
12 30
13 34
14 35
15 39
16 40
17 44
18 45
19 49
20 52
21 54
22 55
23 59
24 60
25 64
26 67
27 69
28 70
29 74
30 77
31 79
32 80
33 84
34 85
35 89
36 90
37 94
38 97
39 99
40 100
41 104
42 107
43 109
44 110
45 114
46 115
47 119
48 120
49 124
50 125
51 129
52 130
53 134
54 135
55 139
56 142
57 144
58 145
59 149
60 150
61 154
62 156
63 159
64 162
65 164
66 165
67 169
68 170
69 174
70 175
71 179
72 182
73 184
74 189
75 189
76 190
77 194
78 195
79 199
80 200
81 204
82 205
83 209
84 210
85 214
86 215
87 219
88 222
89 224
90 226
91 229

In [35]:
error, error_list = KNN(train_new, test_new, knownClass, testClass,3)
print error, error_list
#for i in error_list:
 #   print testClass[i]
#for i in error_list:
#    pokaz(i)

0 [1 4 0]
1 [  4 159 124]
2 [5 6 9]
3 [9 5 7]
4 [10 11 14]
5 [ 14 124 209]
6 [15 19 16]
7 [19 15 17]
8 [20 22 21]
9 [24 20 22]
10 [26 29 27]
11 [29 26 27]
12 [30 32 34]
13 [34  4 32]
14 [35 36 39]
15 [39 35 36]
16 [40 41 42]
17 [ 44 229  40]
18 [45 47 46]
19 [49 45 47]
20 [52 51 50]
21 [54 52 51]
22 [55 57 56]
23 [59 55 57]
24 [60 61 62]
25 [64 60 61]
26 [67 65 22]
27 [69 67 65]
28 [70 72 71]
29 [74 70 72]
30 [77 75 79]
31 [79 77 75]
32 [ 80  81 190]
33 [84 80 81]
34 [85 87 86]
35 [89 87 85]
36 [90 91 94]
37 [94 90 91]
38 [97 95 96]
39 [99 95 97]
40 [100 102 101]
41 [104 100 102]
42 [107 105 106]
43 [109 124 119]
44 [110 112 111]
45 [114 144 112]
46 [115 117 116]
47 [119 115 117]
48 [120 121 122]
49 [124 164 169]
50 [125 129 127]
51 [129 126 125]
52 [130 131 132]
53 [134 131 132]
54 [135 137 136]
55 [139 135 137]
56 [142 140 141]
57 [144 142 140]
58 [145 146 149]
59 [149 145 146]
60 [150 151 154]
61 [154 224 164]
62 [156 159 158]
63 [159 156 157]
64 [162 160 164]
65 [164 162 160]
66 [1